In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [24]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [25]:
# ## Function to create the model and trying different parameters(KerasClassifier)

# def create_model(neurons=32,layers=2):
#     model=Sequential()
#     model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

#     for _ in range(layers-1):
#         model.add(Dense(neurons,activation='relu'))

#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

#     return model

def create_model(num_hidden_layers=1, neurons=64, activation='relu'):
    model = Sequential()
    
    # First layer (input) using 'neurons', 'activation'
    model.add(Dense(neurons, activation=activation, input_shape=(X_train.shape[1],)))
    
    # Adding 'num_hidden_layers - 1' more hidden layers, each with the same neuron count & activation
    for _ in range(num_hidden_layers - 1):
        model.add(Dense(neurons, activation=activation))
    
    # Output layer (binary classification)
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [26]:
# ## Creating a Keras classifier

# model=KerasClassifier(layers=2,neurons=32,build_fn=create_model,verbose=1)

# 4) Wrap your model using KerasClassifier
wrapped_model = KerasClassifier(
    model=create_model,  # pass in your model-builder function
    verbose=0
)

In [27]:
# Defining the grid search parameters
# param_grid = {
#     'neurons': [16, 32, 64, 128],
#     'layers': [1, 2, 3],
#     'epochs': [50, 100]
# }

param_grid = {
    "model__num_hidden_layers": [1, 2, 3],
    "model__neurons": [32, 64, 128],
    "model__activation": ["relu", "tanh"], 
    "batch_size": [32, 64],
    "epochs": [10, 20]
}

In [28]:
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
# grid_result = grid.fit(X_train, y_train)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# 6) Run GridSearchCV on the wrapped model
grid = GridSearchCV(
    estimator=wrapped_model, 
    param_grid=param_grid, 
    cv=3, 
    n_jobs=-1, 
    verbose=1
)

grid_result = grid.fit(X_train, y_train)

print("Best score:", grid_result.best_score_)
print("Best params:", grid_result.best_params_)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


2025-04-06 13:32:53.584754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: At2025-04-06 13:32:53.584755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: At2025-04-06 13:32:53.584775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: At2025-04-06 13:32:53.584767: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
tempting to register factory for plugin cuDNN when one has already been registered
tempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 13:32:53.584826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 13:32:53.584858: 

Best score: 0.8592492142533196
Best params: {'batch_size': 64, 'epochs': 20, 'model__activation': 'tanh', 'model__neurons': 64, 'model__num_hidden_layers': 2}
